In [1]:
from imblearn.over_sampling._smote.base import SMOTE
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pickle
 
class MyClass():
    def __init__(self, param):
        self.param = param
 
def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

X_train = load_object("X_train.pickle")
X_test = load_object("X_test.pickle")
y_train = load_object("y_train.pickle")
y_test = load_object("y_test.pickle")

pipeline_estimators = [('scaling',StandardScaler()),('clf',LogisticRegression())]
pipe = Pipeline(pipeline_estimators)



In [2]:
X_test.head(2)

,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,Humidity,Pressure,...,Thunder / Windy,Thunder and Hail / Windy,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Tornado,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
85699,40.705550,-74.245900,40.703750,-74.246770,0.132,1,81.0,51.852029,58.0,30.05,...,0,0,0,0,0,0,0,0,0,0
29466,32.920590,-96.750610,32.920330,-96.750130,0.033,1,80.6,51.852029,77.0,29.89,...,0,0,0,0,0,0,0,0,0,0
72784,42.915320,-85.540030,42.909430,-85.531420,0.596,1,80.1,51.852029,62.0,30.03,...,0,0,0,0,0,0,0,0,0,0
4991,36.735600,-119.777650,36.727650,-119.783750,0.645,1,71.1,51.852029,31.0,30.08,...,0,0,0,0,0,0,0,0,0,0
2618,37.719510,-122.448220,37.719740,-122.448190,0.016,1,53.1,51.852029,89.0,29.98,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4329,37.363720,-121.898200,37.364380,-121.901870,0.207,1,61.0,51.852029,70.0,29.86,...,0,0,0,0,0,0,0,0,0,0
2768738,32.678709,-96.676421,32.678709,-96.676421,0.000,1,75.7,51.852029,82.0,30.02,...,0,0,0,0,0,0,0,0,0,0
2540539,39.295906,-76.842416,39.296300,-76.832049,0.555,1,33.1,51.852029,96.0,30.09,...,0,0,0,0,0,0,0,0,0,0
2708924,42.920050,-85.760090,42.912010,-85.768760,0.708,1,28.9,21.100000,89.0,30.25,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.metrics import accuracy_score


pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_pred,y_test)

0.8204355594822869

In [4]:
from collections import Counter

Counter(y_pred)

Counter({0.0: 150564, 1.0: 1103})

In [5]:
X_test.loc[[2]]

,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,Humidity,Pressure,...,Thunder / Windy,Thunder and Hail / Windy,Thunder in the Vicinity,Thunderstorm,Thunderstorms and Rain,Tornado,Volcanic Ash,Widespread Dust,Wintry Mix,Wintry Mix / Windy
2,39.10266,-84.52468,39.10209,-84.52396,0.055,1,36.0,51.852029,97.0,29.7,...,0,0,0,0,0,0,0,0,0,0


In [6]:

pred = pipe.predict(X_test.loc[[2]])

In [7]:
pred[0]

0.0

In [8]:
import joblib

joblib.dump(pipe, 'completed_model.joblib')

['completed_model.joblib']